[View in Colaboratory](https://colab.research.google.com/github/neoaksa/tensorflowDemo/blob/master/wordSentimentDemo/wordSentiment.ipynb)

In [0]:
# authenticate google drive
from google.colab import auth
auth.authenticate_user()


In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [4]:
! pip install pydrive
# these classes allow you to request the Google drive API
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def downloadFile(inputfilename,outputfilename):
    downloaded = drive.CreateFile({'id': inputfilename})
    # assume the file is called file.csv and it's located at the root of your drive
    downloaded.GetContentFile(outputfilename)
    
# traning file download
trainingFile = downloadFile("1adyPElLZ118U1aKVEeqrsVNX4b-VoVDm","training.1600000.processed.noemoticon.csv")
# test file download
testingFile = downloadFile("1-6lzGSZ-IkIjYiUULuhpoADPe55aSWcR","testdata.manual.2009.06.14.csv")

In [13]:
# # Download the file we just uploaded.
# #
# # Replace the assignment below with your file ID
# # to download a different file.
# #
# # use native google drive API

# import io
# from googleapiclient.http import MediaIoBaseDownload


# def downloadFile(file_id):
#     request = drive_service.files().get_media(fileId=file_id)
#     downloaded = io.BytesIO()
#     downloader = MediaIoBaseDownload(downloaded, request)
#     done = False
#     while done is False:
#         # _ is a placeholder for a progress object that we ignore.
#         # (Our file is small, so we skip reporting progress.)
#         _, done = downloader.next_chunk()

#     downloaded.seek(0)
#     return downloaded
    
# # traning file download
# trainingFile = downloadFile("1adyPElLZ118U1aKVEeqrsVNX4b-VoVDm")
# # test file download
# testingFile = downloadFile("1-6lzGSZ-IkIjYiUULuhpoADPe55aSWcR")

AttributeError: ignored

In [0]:
# prepare data
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
import pandas as pd

lemmatizer = WordNetLemmatizer()

def init_process(fin, fout):
    outfile = open(fout, 'a')
    with open(fin, buffering=200000, encoding='latin-1') as f:
        try:
            # replace special characters
            for line in f:
                line = line.replace('"', '')
                # the first column is label
                initial_polarity = line.split(',')[0]
                # 0=negative 4=positive
                if initial_polarity == '0':
                    initial_polarity = [1, 0]
                elif initial_polarity == '4':
                    initial_polarity = [0, 1]
                # the last column is input
                tweet = line.split(',')[-1]
                outline = str(initial_polarity) + ':::' + tweet
                outfile.write(outline)
        except Exception as e:
            print(str(e))
    outfile.close()

# save for training and testing dataset
init_process('training.1600000.processed.noemoticon.csv', 'train_set.csv')
init_process('testdata.manual.2009.06.14.csv', 'test_set.csv')
# download and check the pre-processing file
# from google.colab import files
# files.download('test_set.csv')
# files.download('train_set.csv')

In [15]:
# if lack of some componets please run this chuck
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /content/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [16]:

# create lexicon
def create_lexicon(fin):
    lexicon = []
    with open(fin, 'r', buffering=100000, encoding='latin-1') as f:
        try:
            counter = 1
            content = ''
            for line in f:
                counter += 1
                # randomly pick up line for sampling to lexicon
                if (counter / 2500.0).is_integer():
                    tweet = line.split(':::')[1]
                    content += ' ' + tweet
                    words = word_tokenize(content)
                    words = [lemmatizer.lemmatize(i) for i in words]
                    lexicon = list(set(lexicon + words))
                    print(counter, len(lexicon))

        except Exception as e:
            print(str(e))

    with open('lexicon.pickle', 'wb') as f:
        pickle.dump(lexicon, f)

# create lexicon and save to pickle
create_lexicon('train_set.csv')

# convert dataset input to vector
def convert_to_vec(fin, fout, lexicon_pickle):
    # open lexicon
    with open(lexicon_pickle, 'rb') as f:
        lexicon = pickle.load(f)
    outfile = open(fout, 'a')
    with open(fin, buffering=20000, encoding='latin-1') as f:
        counter = 0
        for line in f:
            counter += 1
            label = line.split(':::')[0]
            tweet = line.split(':::')[1]
            # tokenize to array
            current_words = word_tokenize(tweet.lower())
            # lemmatize for each element
            current_words = [lemmatizer.lemmatize(i) for i in current_words]
            # one-hot coding
            features = np.zeros(len(lexicon))
            for word in current_words:
                if word.lower() in lexicon:
                    index_value = lexicon.index(word.lower())
                    # OR DO +=1, test both
                    features[index_value] += 1

            features = list(features)
            outline = str(features) + '::' + str(label) + '\n'
            outfile.write(outline)

        print(counter)


convert_to_vec('test_set.csv', 'processed-test-set.csv', 'lexicon.pickle')


def shuffle_data(fin):
    df = pd.read_csv(fin, error_bad_lines=False)
    df = df.iloc[np.random.permutation(len(df))]
    print(df.head())
    df.to_csv('train_set_shuffled.csv', index=False)


shuffle_data('train_set.csv')

# split the csv into x, y dataset
def create_test_data_pickle(fin):
    feature_sets = []
    labels = []
    counter = 0
    with open(fin, buffering=20000) as f:
        for line in f:
            try:
                features = list(eval(line.split('::')[0]))
                label = list(eval(line.split('::')[1]))

                feature_sets.append(features)
                labels.append(label)
                counter += 1
            except:
                pass
    print(counter)
    feature_sets = np.array(feature_sets)
    labels = np.array(labels)

create_test_data_pickle('processed-test-set.csv')

2500 26
5000 32
7500 41
10000 63
12500 85
15000 89
17500 91
20000 108
22500 113
25000 120
27500 128
30000 141
32500 159
35000 173
37500 177
40000 185
42500 198
45000 212
47500 224
50000 240
52500 252
55000 265
57500 266
60000 268
62500 279
65000 283
67500 290
70000 298
72500 305
75000 320
77500 328
80000 335
82500 344
85000 355
87500 357
90000 360
92500 373
95000 387
97500 398
100000 401
102500 411
105000 417
107500 424
110000 428
112500 437
115000 447
117500 449
120000 452
122500 459
125000 464
127500 465
130000 467
132500 469
135000 479
137500 489
140000 495
142500 499
145000 506
147500 513
150000 527
152500 531
155000 535
157500 539
160000 547
162500 555
165000 559
167500 561
170000 565
172500 570
175000 579
177500 582
180000 587
182500 593
185000 596
187500 599
190000 606
192500 610
195000 620
197500 623
200000 627
202500 629
205000 631
207500 640
210000 643
212500 647
215000 650
217500 658
220000 673
222500 683
225000 688
227500 693
230000 697
232500 699
235000 705
237500 713
2400

777500 1577
780000 1579
782500 1588
785000 1590
787500 1594
790000 1596
792500 1596
795000 1598
797500 1603
800000 1608
802500 1615
805000 1617
807500 1622
810000 1626
812500 1629
815000 1633
817500 1634
820000 1644
822500 1646
825000 1652
827500 1655
830000 1657
832500 1659
835000 1660
837500 1663
840000 1664
842500 1665
845000 1665
847500 1666
850000 1679
852500 1685
855000 1689
857500 1693
860000 1695
862500 1700
865000 1701
867500 1703
870000 1704
872500 1710
875000 1714
877500 1715
880000 1716
882500 1718
885000 1721
887500 1725
890000 1727
892500 1731
895000 1735
897500 1742
900000 1742
902500 1746
905000 1749
907500 1750
910000 1751
912500 1758
915000 1761
917500 1764
920000 1778
922500 1781
925000 1787
927500 1799
930000 1803
932500 1806
935000 1807
937500 1809
940000 1810
942500 1813
945000 1819
947500 1820
950000 1820
952500 1821
955000 1823
957500 1830
960000 1836
962500 1840
965000 1842
967500 1847
970000 1852
972500 1859
975000 1861
977500 1870
980000 1874
982500 1876
9850

1317500 2304
1320000 2305
1322500 2307
1325000 2311
1327500 2315
1330000 2319
1332500 2324
1335000 2328
1337500 2328
1340000 2331
1342500 2331
1345000 2336
1347500 2340
1350000 2345
1352500 2346
1355000 2346
1357500 2349
1360000 2351
1362500 2352
1365000 2357
1367500 2359
1370000 2362
1372500 2365
1375000 2369
1377500 2371
1380000 2375
1382500 2376
1385000 2377
1387500 2381
1390000 2385
1392500 2387
1395000 2388
1397500 2391
1400000 2394
1402500 2398
1405000 2398
1407500 2399
1410000 2405
1412500 2408
1415000 2414
1417500 2415
1420000 2417
1422500 2422
1425000 2423
1427500 2424
1430000 2427
1432500 2428
1435000 2429
1437500 2432
1440000 2433
1442500 2436
1445000 2439
1447500 2439
1450000 2441
1452500 2444
1455000 2447
1457500 2453
1460000 2456
1462500 2457
1465000 2459
1467500 2460
1470000 2468
1472500 2470
1475000 2473
1477500 2480
1480000 2483
1482500 2491
1485000 2498
1487500 2499
1490000 2500
1492500 2501
1495000 2504
1497500 2509
1500000 2513
1502500 2515
1505000 2516
1507500 2517

1752500 2638
1755000 2638
1757500 2638
1760000 2638
1762500 2638
1765000 2638
1767500 2638
1770000 2638
1772500 2638
1775000 2638
1777500 2638
1780000 2638
1782500 2638
1785000 2638
1787500 2638
1790000 2638
1792500 2638
1795000 2638
1797500 2638
1800000 2638
1802500 2638
1805000 2638
1807500 2638
1810000 2638
1812500 2638
1815000 2638
1817500 2638
1820000 2638
1822500 2638
1825000 2638
1827500 2638
1830000 2638
1832500 2638
1835000 2638
1837500 2638
1840000 2638
1842500 2638
1845000 2638
1847500 2638
1850000 2638
1852500 2638
1855000 2638
1857500 2638
1860000 2638
1862500 2638
1865000 2638
1867500 2638
1870000 2638
1872500 2638
1875000 2638
1877500 2638
1880000 2638
1882500 2638
1885000 2638
1887500 2638
1890000 2638
1892500 2638
1895000 2638
1897500 2638
1900000 2638
1902500 2638
1905000 2638
1907500 2638
1910000 2638
1912500 2638
1915000 2638
1917500 2638
1920000 2638
1922500 2638
1925000 2638
1927500 2638
1930000 2638
1932500 2638
1935000 2638
1937500 2638
1940000 2638
1942500 2638

2187500 2638
2190000 2638
2192500 2638
2195000 2638
2197500 2638
2200000 2638
2202500 2638
2205000 2638
2207500 2638
2210000 2638
2212500 2638
2215000 2638
2217500 2638
2220000 2638
2222500 2638
2225000 2638
2227500 2638
2230000 2638
2232500 2638
2235000 2638
2237500 2638
2240000 2638
2242500 2638
2245000 2638
2247500 2638
2250000 2638
2252500 2638
2255000 2638
2257500 2638
2260000 2638
2262500 2638
2265000 2638
2267500 2638
2270000 2638
2272500 2638
2275000 2638
2277500 2638
2280000 2638
2282500 2638
2285000 2638
2287500 2638
2290000 2638
2292500 2638
2295000 2638
2297500 2638
2300000 2638
2302500 2638
2305000 2638
2307500 2638
2310000 2638
2312500 2638
2315000 2638
2317500 2638
2320000 2638
2322500 2638
2325000 2638
2327500 2638
2330000 2638
2332500 2638
2335000 2638
2337500 2638
2340000 2638
2342500 2638
2345000 2638
2347500 2638
2350000 2638
2352500 2638
2355000 2638
2357500 2638
2360000 2638
2362500 2638
2365000 2638
2367500 2638
2370000 2638
2372500 2638
2375000 2638
2377500 2638

...

3560000 2638
3562500 2638
3565000 2638
3567500 2638
3570000 2638
3572500 2638
3575000 2638
3577500 2638
3580000 2638
3582500 2638
3585000 2638
3587500 2638
3590000 2638
3592500 2638
3595000 2638
3597500 2638
3600000 2638
3602500 2638
3605000 2638
3607500 2638
3610000 2638
3612500 2638
3615000 2638
3617500 2638
3620000 2638
3622500 2638
3625000 2638
3627500 2638
3630000 2638
3632500 2638
3635000 2638
3637500 2638
3640000 2638
3642500 2638
3645000 2638
3647500 2638
3650000 2638
3652500 2638
3655000 2638
3657500 2638
3660000 2638
3662500 2638
3665000 2638
3667500 2638
3670000 2638
3672500 2638
3675000 2638
3677500 2638
3680000 2638
3682500 2638
3685000 2638
3687500 2638
3690000 2638
3692500 2638
3695000 2638
3697500 2638
3700000 2638
3702500 2638
3705000 2638
3707500 2638
3710000 2638
3712500 2638
3715000 2638
3717500 2638
3720000 2638
3722500 2638
3725000 2638
3727500 2638
3730000 2638
3732500 2638
3735000 2638
3737500 2638
3740000 2638
3742500 2638
3745000 2638
3747500 2638
3750000 2638

3867500 2638
3870000 2638
3872500 2638
3875000 2638
3877500 2638
3880000 2638
3882500 2638
3885000 2638
3887500 2638
3890000 2638
3892500 2638
3895000 2638
3897500 2638
3900000 2638
3902500 2638
3905000 2638
3907500 2638
3910000 2638
3912500 2638
3915000 2638
3917500 2638
3920000 2638
3922500 2638
3925000 2638
3927500 2638
3930000 2638
3932500 2638
3935000 2638
3937500 2638
3940000 2638
3942500 2638
3945000 2638
3947500 2638
3950000 2638
3952500 2638
3955000 2638
3957500 2638
3960000 2638
3962500 2638
3965000 2638
3967500 2638
3970000 2638
3972500 2638
3975000 2638
3977500 2638
3980000 2638
3982500 2638
3985000 2638
3987500 2638
3990000 2638
3992500 2638
3995000 2638
3997500 2638
4000000 2638
4002500 2638
4005000 2638
4007500 2638
4010000 2638
4012500 2638
4015000 2638
4017500 2638
4020000 2638
4022500 2638
4025000 2638
4027500 2638
4030000 2638
4032500 2638
4035000 2638
4037500 2638
4040000 2638
4042500 2638
4045000 2638
4047500 2638
4050000 2638
4052500 2638
4055000 2638
4057500 2638

4175000 2638
4177500 2638
4180000 2638
4182500 2638
4185000 2638
4187500 2638
4190000 2638
4192500 2638
4195000 2638
4197500 2638
4200000 2638
4202500 2638
4205000 2638
4207500 2638
4210000 2638
4212500 2638
4215000 2638
4217500 2638
4220000 2638
4222500 2638
4225000 2638
4227500 2638
4230000 2638
4232500 2638
4235000 2638
4237500 2638
4240000 2638
4242500 2638
4245000 2638
4247500 2638
4250000 2638
4252500 2638
4255000 2638
4257500 2638
4260000 2638
4262500 2638
4265000 2638
4267500 2638
4270000 2638
4272500 2638
4275000 2638
4277500 2638
4280000 2638
4282500 2638
4285000 2638
4287500 2638
4290000 2638
4292500 2638
4295000 2638
4297500 2638
4300000 2638
4302500 2638
4305000 2638
4307500 2638
4310000 2638
4312500 2638
4315000 2638
4317500 2638
4320000 2638
4322500 2638
4325000 2638
4327500 2638
4330000 2638
4332500 2638
4335000 2638
4337500 2638
4340000 2638
4342500 2638
4345000 2638
4347500 2638
4350000 2638
4352500 2638
4355000 2638
4357500 2638
4360000 2638
4362500 2638
4365000 2638

4482500 2638
4485000 2638
4487500 2638
4490000 2638
4492500 2638
4495000 2638
4497500 2638
4500000 2638
4502500 2638
4505000 2638
4507500 2638
4510000 2638
4512500 2638
4515000 2638
4517500 2638
4520000 2638
4522500 2638
4525000 2638
4527500 2638
4530000 2638
4532500 2638
4535000 2638
4537500 2638
4540000 2638
4542500 2638
4545000 2638
4547500 2638
4550000 2638
4552500 2638
4555000 2638
4557500 2638
4560000 2638
4562500 2638
4565000 2638
4567500 2638
4570000 2638
4572500 2638
4575000 2638
4577500 2638
4580000 2638
4582500 2638
4585000 2638
4587500 2638
4590000 2638
4592500 2638
4595000 2638
4597500 2638
4600000 2638
4602500 2638
4605000 2638
4607500 2638
4610000 2638
4612500 2638
4615000 2638
4617500 2638
4620000 2638
4622500 2638
4625000 2638
4627500 2638
4630000 2638
4632500 2638
4635000 2638
4637500 2638
4640000 2638
4642500 2638
4645000 2638
4647500 2638
4650000 2638
4652500 2638
4655000 2638
4657500 2638
4660000 2638
4662500 2638
4665000 2638
4667500 2638
4670000 2638
4672500 2638

4790000 2638
4792500 2638
4795000 2638
4797500 2638
4800000 2638
1494
         [1  \
1672449  [1   
2829874  [0   
64558    [1   
2031771  [1   
4316972  [0   

         0]::: that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  
1672449   0]:::I got 3 extra followers overnight but th...                             
2829874   1]:::@DENISE_RICHARDS HI Denise. your daughte...                             
64558     0]:::@breedimetria awwww! Give that princess ...                             
2031771   0]:::@Kriptic just done AS level so i still h...                             
4316972                                     1]:::@jipjean                              
1077


In [19]:
# the following step will take very long time, so we need to check if we use GPU
tf.test.gpu_device_name()

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13837335193735682741, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11287966516
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4674334258966792621
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
"""
this snippet is used for build ANN net work, train the model and test it
before you run this snippet, you should run word2vec.py first which transfer
training and testing data to one-hot code vector.
"""

import tensorflow as tf
import pickle
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from functools import singledispatch

lemmatizer = WordNetLemmatizer()


# setting ANN variables
with open("lexicon.pickle", 'rb') as f:
    lexicon = pickle.load(f)
input_size = len(lexicon)
output_size = 2
structure = [input_size,500,500,output_size]
x = tf.placeholder(dtype='float',shape=[None,structure[0]])
y = tf.placeholder(dtype='float')
batch_size = 100
epoch_max = 10
total_batches = int(1600000 / batch_size)

# feedforward Model
def netural_network_model(x):
    # l is output value from active function
    l = 0
    l_prev = 0
    for i in range(1,len(structure)):
        # create each layer structure
        hidden_layer = {'weight':tf.Variable(tf.random_normal([structure[i-1],structure[i]])),
                        'biases':tf.Variable(tf.random_normal([structure[i]]))}
        # input layer--> first hidden layer
        if i == 1:
            l = tf.add(tf.matmul(x, hidden_layer['weight']),hidden_layer['biases'])
            l = tf.nn.relu(l)
            l_prev = l
        # --->output layer without active function
        elif i == len(structure)-1:
            l = tf.add(tf.matmul(l_prev, hidden_layer['weight']), hidden_layer['biases'])
        # hidden layer ---> hidden layer with Relu active function
        else:
            l = tf.add(tf.matmul(l_prev, hidden_layer['weight']), hidden_layer['biases'])
            l = tf.nn.relu(l)
            l_prev = l
    return l


tf_log = 'tf.log'


def train_neural_network(x):
    # prediction, cost and gradient decrease
    prediction = netural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        saver = tf.train.Saver()
        try:
            epoch = int(open(tf_log, 'r').read().split('\n')[-2]) + 1
            print('STARTING:', epoch)
        except:
            epoch = 1

        while epoch <= epoch_max:
            # continue the session data
            if epoch != 1:
                saver.restore(sess, "model.ckpt")
            epoch_loss = 1
            # load the prepared lexicon
            with open('lexicon.pickle', 'rb') as f:
                lexicon = pickle.load(f)
            # load shuffled traning dataset
            with open('train_set_shuffled.csv', buffering=20000, encoding='latin-1') as f:
                batch_x = []
                batch_y = []
                batches_run = 0
                for line in f:
                    label = line.split(':::')[0]
                    tweet = line.split(':::')[1]
                    current_words = word_tokenize(tweet.lower())
                    current_words = [lemmatizer.lemmatize(i) for i in current_words]

                    features = np.zeros(len(lexicon))

                    for word in current_words:
                        if word.lower() in lexicon:
                            index_value = lexicon.index(word.lower())
                            # OR DO +=1, test both
                            features[index_value] += 1
                    line_x = list(features)
                    line_y = eval(label)
                    batch_x.append(line_x)
                    batch_y.append(line_y)
                    if len(batch_x) >= batch_size:
                        _, c = sess.run([optimizer, cost], feed_dict={x: np.array(batch_x),
                                                                      y: np.array(batch_y)})
                        epoch_loss += c
                        batch_x = []
                        batch_y = []
                        batches_run += 1
                        print('Batch run:', batches_run, '/', total_batches, '| Epoch:', epoch, '| Batch Loss:', c, )
            # save session for each epoch
            saver.save(sess, "model.ckpt")
            print('Epoch', epoch, 'completed out of', epoch_max, 'loss:', epoch_loss)
            with open(tf_log, 'a') as f:
                f.write(str(epoch) + '\n')
            epoch += 1


train_neural_network(x)

@singledispatch
def test_neural_network():
    prediction = netural_network_model(x)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        saver = tf.train.Saver()
        # load the traning session for model
        try:
            saver.restore(sess, "model.ckpt")
        except Exception as e:
            print(str(e))

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        # be ware there is no optimizer, since we dont need backprepergate
        feature_sets = []
        labels = []
        counter = 0
        # load test dataset
        with open('processed-test-set.csv', buffering=20000) as f:
            for line in f:
                try:
                    features = list(eval(line.split('::')[0]))
                    label = list(eval(line.split('::')[1]))
                    feature_sets.append(features)
                    labels.append(label)
                    counter += 1
                except:
                    pass
        print('Tested', counter, 'samples.')
        test_x = np.array(feature_sets)
        test_y = np.array(labels)
        print('Accuracy:', accuracy.eval({x: test_x, y: test_y}))

test_neural_network()

@test_neural_network.register(str)
def test_neural_network(input_data):
    prediction = netural_network_model(x)
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        saver = tf.train.Saver()
        with open('lexicon.pickle', 'rb') as f:
            lexicon = pickle.load(f)
        # load the session
        try:
            saver.restore(sess, "model.ckpt")
        except Exception as e:
            print(str(e))
        # lemmatize the input data
        current_words = word_tokenize(input_data.lower())
        current_words = [lemmatizer.lemmatize(i) for i in current_words]
        # one hot coding
        features = np.zeros(len(lexicon))
        for word in current_words:
            if word.lower() in lexicon:
                index_value = lexicon.index(word.lower())
                # OR DO +=1, test both
                features[index_value] += 1
        features = np.array(list(features))
        # pos: [1,0] , argmax: 0
        # neg: [0,1] , argmax: 1
        result = (sess.run(tf.argmax(prediction.eval(feed_dict={x: [features]}), 1)))
        if result[0] == 0:
            print('Positive:', input_data)
        elif result[0] == 1:
            print('Negative:', input_data)

test_neural_network("I hate you")

Batch run: 1 / 16000 | Epoch: 1 | Batch Loss: 525.94354
Batch run: 2 / 16000 | Epoch: 1 | Batch Loss: 407.80594
Batch run: 3 / 16000 | Epoch: 1 | Batch Loss: 451.53613
Batch run: 4 / 16000 | Epoch: 1 | Batch Loss: 394.14368
Batch run: 5 / 16000 | Epoch: 1 | Batch Loss: 221.10367
Batch run: 6 / 16000 | Epoch: 1 | Batch Loss: 325.36435
Batch run: 7 / 16000 | Epoch: 1 | Batch Loss: 254.6326
Batch run: 8 / 16000 | Epoch: 1 | Batch Loss: 310.88193
Batch run: 9 / 16000 | Epoch: 1 | Batch Loss: 336.89117
Batch run: 10 / 16000 | Epoch: 1 | Batch Loss: 296.61465
Batch run: 11 / 16000 | Epoch: 1 | Batch Loss: 358.58118
Batch run: 12 / 16000 | Epoch: 1 | Batch Loss: 353.42133
Batch run: 13 / 16000 | Epoch: 1 | Batch Loss: 346.50632
Batch run: 14 / 16000 | Epoch: 1 | Batch Loss: 284.3027
Batch run: 15 / 16000 | Epoch: 1 | Batch Loss: 297.7537
Batch run: 16 / 16000 | Epoch: 1 | Batch Loss: 201.96692
Batch run: 17 / 16000 | Epoch: 1 | Batch Loss: 336.1519
Batch run: 18 / 16000 | Epoch: 1 | Batch Los

Batch run: 93 / 16000 | Epoch: 1 | Batch Loss: 233.75082
Batch run: 94 / 16000 | Epoch: 1 | Batch Loss: 172.19969
Batch run: 95 / 16000 | Epoch: 1 | Batch Loss: 187.74193
Batch run: 96 / 16000 | Epoch: 1 | Batch Loss: 157.91351
Batch run: 97 / 16000 | Epoch: 1 | Batch Loss: 128.64888
Batch run: 98 / 16000 | Epoch: 1 | Batch Loss: 167.81708
Batch run: 99 / 16000 | Epoch: 1 | Batch Loss: 160.50043
Batch run: 100 / 16000 | Epoch: 1 | Batch Loss: 111.732445
Batch run: 101 / 16000 | Epoch: 1 | Batch Loss: 142.1079
Batch run: 102 / 16000 | Epoch: 1 | Batch Loss: 160.75761
Batch run: 103 / 16000 | Epoch: 1 | Batch Loss: 167.67924
Batch run: 104 / 16000 | Epoch: 1 | Batch Loss: 194.79715
Batch run: 105 / 16000 | Epoch: 1 | Batch Loss: 166.83243
Batch run: 106 / 16000 | Epoch: 1 | Batch Loss: 125.62015
Batch run: 107 / 16000 | Epoch: 1 | Batch Loss: 148.25693
Batch run: 108 / 16000 | Epoch: 1 | Batch Loss: 131.31586
Batch run: 109 / 16000 | Epoch: 1 | Batch Loss: 213.29247
Batch run: 110 / 1600

Batch run: 186 / 16000 | Epoch: 1 | Batch Loss: 103.912704
Batch run: 187 / 16000 | Epoch: 1 | Batch Loss: 120.5063
Batch run: 188 / 16000 | Epoch: 1 | Batch Loss: 99.47124
Batch run: 189 / 16000 | Epoch: 1 | Batch Loss: 110.95666
Batch run: 190 / 16000 | Epoch: 1 | Batch Loss: 145.01382
Batch run: 191 / 16000 | Epoch: 1 | Batch Loss: 81.71035
Batch run: 192 / 16000 | Epoch: 1 | Batch Loss: 157.11636
Batch run: 193 / 16000 | Epoch: 1 | Batch Loss: 127.32491
Batch run: 194 / 16000 | Epoch: 1 | Batch Loss: 136.14267
Batch run: 195 / 16000 | Epoch: 1 | Batch Loss: 106.025314
Batch run: 196 / 16000 | Epoch: 1 | Batch Loss: 147.88818
Batch run: 197 / 16000 | Epoch: 1 | Batch Loss: 138.24292
Batch run: 198 / 16000 | Epoch: 1 | Batch Loss: 115.9648
Batch run: 199 / 16000 | Epoch: 1 | Batch Loss: 125.593285
Batch run: 200 / 16000 | Epoch: 1 | Batch Loss: 130.06047
Batch run: 201 / 16000 | Epoch: 1 | Batch Loss: 139.49129
Batch run: 202 / 16000 | Epoch: 1 | Batch Loss: 128.90703
Batch run: 203 

Batch run: 282 / 16000 | Epoch: 1 | Batch Loss: 100.51152
Batch run: 283 / 16000 | Epoch: 1 | Batch Loss: 138.18178
Batch run: 284 / 16000 | Epoch: 1 | Batch Loss: 114.1297
Batch run: 285 / 16000 | Epoch: 1 | Batch Loss: 110.12441
Batch run: 286 / 16000 | Epoch: 1 | Batch Loss: 113.500626
Batch run: 287 / 16000 | Epoch: 1 | Batch Loss: 141.35852
Batch run: 288 / 16000 | Epoch: 1 | Batch Loss: 105.02503
Batch run: 289 / 16000 | Epoch: 1 | Batch Loss: 111.72718
Batch run: 290 / 16000 | Epoch: 1 | Batch Loss: 97.92895
Batch run: 291 / 16000 | Epoch: 1 | Batch Loss: 79.86824
Batch run: 292 / 16000 | Epoch: 1 | Batch Loss: 90.22047
Batch run: 293 / 16000 | Epoch: 1 | Batch Loss: 86.08681
Batch run: 294 / 16000 | Epoch: 1 | Batch Loss: 117.326164
Batch run: 295 / 16000 | Epoch: 1 | Batch Loss: 85.42441
Batch run: 296 / 16000 | Epoch: 1 | Batch Loss: 119.7751
Batch run: 297 / 16000 | Epoch: 1 | Batch Loss: 91.94799
Batch run: 298 / 16000 | Epoch: 1 | Batch Loss: 136.73639
Batch run: 299 / 160

Batch run: 375 / 16000 | Epoch: 1 | Batch Loss: 57.34038
Batch run: 376 / 16000 | Epoch: 1 | Batch Loss: 85.239876
Batch run: 377 / 16000 | Epoch: 1 | Batch Loss: 135.23848
Batch run: 378 / 16000 | Epoch: 1 | Batch Loss: 129.34134
Batch run: 379 / 16000 | Epoch: 1 | Batch Loss: 83.42411
Batch run: 380 / 16000 | Epoch: 1 | Batch Loss: 112.31674
Batch run: 381 / 16000 | Epoch: 1 | Batch Loss: 121.92129
Batch run: 382 / 16000 | Epoch: 1 | Batch Loss: 58.599316
Batch run: 383 / 16000 | Epoch: 1 | Batch Loss: 104.27166
Batch run: 384 / 16000 | Epoch: 1 | Batch Loss: 84.194275
Batch run: 385 / 16000 | Epoch: 1 | Batch Loss: 104.458244
Batch run: 386 / 16000 | Epoch: 1 | Batch Loss: 103.209206
Batch run: 387 / 16000 | Epoch: 1 | Batch Loss: 76.265915
Batch run: 388 / 16000 | Epoch: 1 | Batch Loss: 74.87379
Batch run: 389 / 16000 | Epoch: 1 | Batch Loss: 138.18994
Batch run: 390 / 16000 | Epoch: 1 | Batch Loss: 122.522385
Batch run: 391 / 16000 | Epoch: 1 | Batch Loss: 117.358925
Batch run: 39

...

Batch run: 12832 / 16000 | Epoch: 1 | Batch Loss: 2.387868
Batch run: 12833 / 16000 | Epoch: 1 | Batch Loss: 2.9127445
Batch run: 12834 / 16000 | Epoch: 1 | Batch Loss: 2.1036172
Batch run: 12835 / 16000 | Epoch: 1 | Batch Loss: 4.420166
Batch run: 12836 / 16000 | Epoch: 1 | Batch Loss: 2.083419
Batch run: 12837 / 16000 | Epoch: 1 | Batch Loss: 3.76045
Batch run: 12838 / 16000 | Epoch: 1 | Batch Loss: 3.348117
Batch run: 12839 / 16000 | Epoch: 1 | Batch Loss: 3.0075219
Batch run: 12840 / 16000 | Epoch: 1 | Batch Loss: 1.8852391
Batch run: 12841 / 16000 | Epoch: 1 | Batch Loss: 4.265461
Batch run: 12842 / 16000 | Epoch: 1 | Batch Loss: 2.3479676
Batch run: 12843 / 16000 | Epoch: 1 | Batch Loss: 3.4205835
Batch run: 12844 / 16000 | Epoch: 1 | Batch Loss: 3.907738
Batch run: 12845 / 16000 | Epoch: 1 | Batch Loss: 4.2960796
Batch run: 12846 / 16000 | Epoch: 1 | Batch Loss: 3.1412542
Batch run: 12847 / 16000 | Epoch: 1 | Batch Loss: 3.026
Batch run: 12848 / 16000 | Epoch: 1 | Batch Loss: 1.

Batch run: 12920 / 16000 | Epoch: 1 | Batch Loss: 2.7777827
Batch run: 12921 / 16000 | Epoch: 1 | Batch Loss: 2.2369692
Batch run: 12922 / 16000 | Epoch: 1 | Batch Loss: 2.6995459
Batch run: 12923 / 16000 | Epoch: 1 | Batch Loss: 6.6758265
Batch run: 12924 / 16000 | Epoch: 1 | Batch Loss: 2.6421201
Batch run: 12925 / 16000 | Epoch: 1 | Batch Loss: 5.907564
Batch run: 12926 / 16000 | Epoch: 1 | Batch Loss: 2.1783195
Batch run: 12927 / 16000 | Epoch: 1 | Batch Loss: 5.1189976
Batch run: 12928 / 16000 | Epoch: 1 | Batch Loss: 4.2476945
Batch run: 12929 / 16000 | Epoch: 1 | Batch Loss: 5.5958424
Batch run: 12930 / 16000 | Epoch: 1 | Batch Loss: 4.504362
Batch run: 12931 / 16000 | Epoch: 1 | Batch Loss: 6.303974
Batch run: 12932 / 16000 | Epoch: 1 | Batch Loss: 8.576592
Batch run: 12933 / 16000 | Epoch: 1 | Batch Loss: 4.7910304
Batch run: 12934 / 16000 | Epoch: 1 | Batch Loss: 6.71583
Batch run: 12935 / 16000 | Epoch: 1 | Batch Loss: 3.035652
Batch run: 12936 / 16000 | Epoch: 1 | Batch Los

Batch run: 13010 / 16000 | Epoch: 1 | Batch Loss: 4.1960387
Batch run: 13011 / 16000 | Epoch: 1 | Batch Loss: 4.915544
Batch run: 13012 / 16000 | Epoch: 1 | Batch Loss: 3.3896015
Batch run: 13013 / 16000 | Epoch: 1 | Batch Loss: 5.866324
Batch run: 13014 / 16000 | Epoch: 1 | Batch Loss: 2.8172584
Batch run: 13015 / 16000 | Epoch: 1 | Batch Loss: 5.2400856
Batch run: 13016 / 16000 | Epoch: 1 | Batch Loss: 2.5336542
Batch run: 13017 / 16000 | Epoch: 1 | Batch Loss: 5.299704
Batch run: 13018 / 16000 | Epoch: 1 | Batch Loss: 3.2392
Batch run: 13019 / 16000 | Epoch: 1 | Batch Loss: 3.631984
Batch run: 13020 / 16000 | Epoch: 1 | Batch Loss: 6.8180375
Batch run: 13021 / 16000 | Epoch: 1 | Batch Loss: 3.4536788
Batch run: 13022 / 16000 | Epoch: 1 | Batch Loss: 2.5336654
Batch run: 13023 / 16000 | Epoch: 1 | Batch Loss: 2.0540996
Batch run: 13024 / 16000 | Epoch: 1 | Batch Loss: 3.5701232
Batch run: 13025 / 16000 | Epoch: 1 | Batch Loss: 3.3668833
Batch run: 13026 / 16000 | Epoch: 1 | Batch Los

Batch run: 13097 / 16000 | Epoch: 1 | Batch Loss: 2.1514351
Batch run: 13098 / 16000 | Epoch: 1 | Batch Loss: 5.677617
Batch run: 13099 / 16000 | Epoch: 1 | Batch Loss: 5.0727077
Batch run: 13100 / 16000 | Epoch: 1 | Batch Loss: 4.6855526
Batch run: 13101 / 16000 | Epoch: 1 | Batch Loss: 5.0694103
Batch run: 13102 / 16000 | Epoch: 1 | Batch Loss: 3.1162047
Batch run: 13103 / 16000 | Epoch: 1 | Batch Loss: 5.1991534
Batch run: 13104 / 16000 | Epoch: 1 | Batch Loss: 3.2608979
Batch run: 13105 / 16000 | Epoch: 1 | Batch Loss: 4.4186683
Batch run: 13106 / 16000 | Epoch: 1 | Batch Loss: 3.4060752
Batch run: 13107 / 16000 | Epoch: 1 | Batch Loss: 2.6454012
Batch run: 13108 / 16000 | Epoch: 1 | Batch Loss: 3.7762973
Batch run: 13109 / 16000 | Epoch: 1 | Batch Loss: 2.7687967
Batch run: 13110 / 16000 | Epoch: 1 | Batch Loss: 5.1998425
Batch run: 13111 / 16000 | Epoch: 1 | Batch Loss: 2.6061916
Batch run: 13112 / 16000 | Epoch: 1 | Batch Loss: 4.699453
Batch run: 13113 / 16000 | Epoch: 1 | Batc

 | Batch Loss: 3.319603
Batch run: 13188 / 16000 | Epoch: 1 | Batch Loss: 1.959739
Batch run: 13189 / 16000 | Epoch: 1 | Batch Loss: 3.7688568
Batch run: 13190 / 16000 | Epoch: 1 | Batch Loss: 2.3854177
Batch run: 13191 / 16000 | Epoch: 1 | Batch Loss: 1.9798449
Batch run: 13192 / 16000 | Epoch: 1 | Batch Loss: 2.8649168
Batch run: 13193 / 16000 | Epoch: 1 | Batch Loss: 3.4069588
Batch run: 13194 / 16000 | Epoch: 1 | Batch Loss: 2.50012
Batch run: 13195 / 16000 | Epoch: 1 | Batch Loss: 3.4051368
Batch run: 13196 / 16000 | Epoch: 1 | Batch Loss: 2.9112432
Batch run: 13197 / 16000 | Epoch: 1 | Batch Loss: 2.4248636
Batch run: 13198 / 16000 | Epoch: 1 | Batch Loss: 2.4613135
Batch run: 13199 / 16000 | Epoch: 1 | Batch Loss: 1.6443249
Batch run: 13200 / 16000 | Epoch: 1 | Batch Loss: 2.384141
Batch run: 13201 / 16000 | Epoch: 1 | Batch Loss: 3.0887504
Batch run: 13202 / 16000 | Epoch: 1 | Batch Loss: 2.4138706
Batch run: 13203 / 16000 | Epoch: 1 | Batch Loss: 2.7914736
Batch run: 13204 / 1